This notebook shows how to use Elastic Search to index and search through data. We will use a dataset called CMU Book summaries [dataset](http://www.cs.cmu.edu/~dbamman/booksummaries.html). Alternateively, the dataset's link can be found in the `BookSummaries_Link.md` file under the Data folder in Ch7. 

For this code to work, elastic search instance has to be running in the background. 
For this you need to follow these steps :

Linux :

   1. Go to the elasticsearch-X.Y.Z/bin folder on your machine
   2. Run ./elasticsearch.  
    
Windows :

   1.  Download the latest [release](https://www.elastic.co/guide/en/elasticsearch/reference/current/windows.html)
   2.  Run .\bin\elasticsearch.bat
   
[ElasticSearch Documentation](https://www.elastic.co/guide/index.html)
    
You should now be able to access this instance on localhost:9200



In [2]:
!pip install elasticsearch

     |████████████████████████████████| 385 kB 4.9 MB/s 


In [3]:
from elasticsearch import Elasticsearch 
from datetime import datetime

In [6]:

#Here's how I run ElasticSearch on Colab

!wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.0.0-linux-x86_64.tar.gz -q
!tar -xzf elasticsearch-7.0.0-linux-x86_64.tar.gz
!chown -R daemon:daemon elasticsearch-7.0.0

In [7]:
# start server
import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.0.0/bin/elasticsearch'], 
                  stdout=PIPE, stderr=STDOUT,
                  preexec_fn=lambda: os.setuid(1)  # as daemon
                 )

In [ ]:
# wait a bit then test
#!curl -X GET "localhost:9200/"

In [8]:
#elastic search instance has to be running on the machine. Default port is 9200. 

#Call the Elastic Search instance, and delete any pre-existing index
es=Elasticsearch([{'host':'localhost','port':9200}])
if es.indices.exists(index="myindex"):
    es.indices.delete(index='myindex', ignore=[400, 404]) #Deleting existing index for now 

In [11]:
#Build an index from booksummaries dataset. I am using only 500 documents for now.
path = "/content/booksummaries.txt" #Add your path.
count = 1
for line in open(path):
    fields = line.split("\t")
    doc = {'id' : fields[0],
            'title': fields[2],
            'author': fields[3],
            'summary': fields[6]
          }

    res = es.index(index="myindex", id=fields[0], body=doc)
    count = count+1
    if count%100 == 0:
        print("indexed 100 documents")
    if count == 501:
        break

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: The 'body' parameter is deprecated for the 'index' API and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  if sys.path[0] == '':


indexed 100 documents
indexed 100 documents
indexed 100 documents
indexed 100 documents
indexed 100 documents


In [15]:
len(res)

8

In [16]:
#Check to see how big is the index
res = es.search(index="myindex", body={"query": {"match_all": {}}})
print("Your index has %d entries" % res['hits']['total']['value'])

Your index has 500 entries


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  


In [ ]:
#Try a test query. The query searches "summary" field which contains the text
#and does a full text query on that field.
res = es.search(index="myindex", body={"query": {"match": {"summary": "animal"}}})
print("Your search returned %d results." % res['hits']['total']['value'])

Your search returned 16 results.


In [ ]:
#Printing the title field and summary field's first 100 characters for 2nd result
print(res["hits"]["hits"][2]["_source"]["title"])
print(res["hits"]["hits"][2]["_source"]["summary"][:100])


Dead Air
 The first person narrative begins on 11 September 2001, and Banks uses the protagonist's conversati


In [ ]:
#match query considers both exact matches, and fuzzy matches and works as a OR query. 
#match_phrase looks for exact matches.
while True:
    query = input("Enter your search query: ")
    if query == "STOP":
        break
    res = es.search(index="myindex", body={"query": {"match_phrase": {"summary": query}}})
    print("Your search returned %d results:" % res['hits']['total']['value'])
    for hit in res["hits"]["hits"]:
        print(hit["_source"]["title"])
        #to get a snippet 100 characters before and after the match
        loc = hit["_source"]["summary"].lower().index(query)
        print(hit["_source"]["summary"][:100])
        print(hit["_source"]["summary"][loc-100:loc+100])

    

Enter your search query: countess
Your search returned 7 results:
All's Well That Ends Well
71
 Helena, the orphan daughter of a famous physician, is the ward of the Countess of Rousillon, and ho

The Last Man
904
 Mary Shelley states in the introduction that in 1818 she discovered, in the Sibyl's cave near Naple
ng leaves the throne, the monarchy come to an end and a republic is created. When the king dies the Countess attempts to raise their son, Adrian, to reclaim the throne, but Adrian opposes his mother a
The Luck of Barry Lyndon
2624
 Redmond Barry of Bally Barry, born to a genteel but ruined Irish family, fancies himself a gentlema
chy, where they win considerable sums of money and Redmond cleverly sets up a plan to marry a young countess of some means. Again, fortune turns against him, and a series of circumstances undermines h
Carmilla
2554
 The story is presented by Le Fanu as part of the casebook of Dr Hesselius, whose departures from me
ily heirloom restored portraits arriv